<a href="https://colab.research.google.com/github/kowook137/Model_Discriminator_module/blob/youn/FaceModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 1. 필수 라이브러리
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models  # models 추가
import pandas as pd
import os
from PIL import Image
from torch.cuda.amp import GradScaler, autocast


# 2. Kaggle API 설정
!pip install -q kaggle
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 3. 데이터셋 다운로드 및 압축 해제
!kaggle datasets download -d mehmoodsheikh/fairface-dataset --force
!unzip -oq fairface-dataset.zip -d fairface

# 4. 데이터 경로 확인
base_path = '/content/fairface/FairFace'
print("파일 구조:", os.listdir(base_path))

# 5. 커스텀 데이터셋 클래스 (인종 전용)
class FairFaceDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.race_map = {
            'White':0, 'Black':1, 'Latino_Hispanic':2,
            'East Asian':3, 'Southeast Asian':4,
            'Indian':5, 'Middle Eastern':6
        }

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.labels.iloc[idx]['file'])
        image = Image.open(img_path).convert('RGB')
        race = self.race_map[self.labels.iloc[idx]['race']]
        if self.transform:
            image = self.transform(image)
        return image, race

# 6. 데이터 전처리 (이미지 크기 224x224로 변경)
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),  # 224x224로 변경
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(224),             # 224x224로 변경
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 7. 데이터셋 생성
train_data = FairFaceDataset(
    csv_file=os.path.join(base_path, 'fairface_label_train.csv'),
    img_dir=base_path,
    transform=train_transforms
)

test_data = FairFaceDataset(
    csv_file=os.path.join(base_path, 'fairface_label_val.csv'),
    img_dir=base_path,
    transform=test_transforms
)

# 8. 데이터로더 (배치 크기 조정)
BATCH_SIZE = 32  # GPU 메모리 고려하여 조정
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# 9. 데이터셋 정보 출력
print(f"Train samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")
print("Race Class mapping:", train_data.race_map)

# 10. 전이 학습 모델 정의 (ResNet-18)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18(pretrained=True)  # 사전 학습된 모델 로드

# 마지막 레이어 수정
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # 7개 클래스 출력
model = model.to(device)

# 11. 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 12. 학습 루프
EPOCHS = 5  # 전이 학습은 적은 에포크로도 충분
scaler = GradScaler()  # 혼합 정밀도 학습
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # 혼합 정밀도 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        if i % 50 == 49:  # 출력 주기 줄임
            print(f'Epoch {epoch+1}, Batch {i+1}: Loss {running_loss/50:.3f}')
            running_loss = 0.0
# 13. 테스트
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for (images, labels) in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Saving kaggle.json to kaggle (4).json
Dataset URL: https://www.kaggle.com/datasets/mehmoodsheikh/fairface-dataset
License(s): MIT
 96% 530M/550M [00:01<00:00, 248MB/s]
100% 550M/550M [00:01<00:00, 357MB/s]
파일 구조: ['fairface_label_val.csv', 'fairface_label_train.csv', 'train', 'val']
Train samples: 86744
Test samples: 10954
Race Class mapping: {'White': 0, 'Black': 1, 'Latino_Hispanic': 2, 'East Asian': 3, 'Southeast Asian': 4, 'Indian': 5, 'Middle Eastern': 6}


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-5-e4300cbcedbd>:106: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # 혼합 정밀도 학습
<ipython-input-5-e4300cbcedbd>:116: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():

Epoch 1, Batch 50: Loss 2.040
Epoch 1, Batch 100: Loss 1.948
Epoch 1, Batch 150: Loss 1.888
Epoch 1, Batch 200: Loss 1.896
Epoch 1, Batch 250: Loss 1.864
Epoch 1, Batch 300: Loss 1.866
Epoch 1, Batch 350: Loss 1.868
Epoch 1, Batch 400: Loss 1.819
Epoch 1, Batch 450: Loss 1.862
Epoch 1, Batch 500: Loss 1.816
Epoch 1, Batch 550: Loss 1.822
Epoch 1, Batch 600: Loss 1.813
Epoch 1, Batch 650: Loss 1.765
Epoch 1, Batch 700: Loss 1.769
Epoch 1, Batch 750: Loss 1.760
Epoch 1, Batch 800: Loss 1.754
Epoch 1, Batch 850: Loss 1.724
Epoch 1, Batch 900: Loss 1.726
Epoch 1, Batch 950: Loss 1.700
Epoch 1, Batch 1000: Loss 1.690
Epoch 1, Batch 1050: Loss 1.667
Epoch 1, Batch 1100: Loss 1.696
Epoch 1, Batch 1150: Loss 1.643
Epoch 1, Batch 1200: Loss 1.657
Epoch 1, Batch 1250: Loss 1.665
Epoch 1, Batch 1300: Loss 1.666
Epoch 1, Batch 1350: Loss 1.617
Epoch 1, Batch 1400: Loss 1.665
Epoch 1, Batch 1450: Loss 1.634
Epoch 1, Batch 1500: Loss 1.617
Epoch 1, Batch 1550: Loss 1.601
Epoch 1, Batch 1600: Loss 1.

인종 정확도: 57.84%
